In [1]:
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt    

import sklearn 
from sklearn import preprocessing, metrics, naive_bayes, pipeline, feature_extraction
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction import text
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix

from sklearn import neighbors, tree, ensemble, naive_bayes, svm
# *** KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
import sklearn.metrics


import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import metrics

from sklearn.model_selection import train_test_split


from math import floor


In [47]:
#Input function
def text_prediction(text, clf):
  text = [text]
  prediction = clf.predict(text)

  if (prediction[0] == 1):
    print(f'"{text}" is probably a good review')
  
  else:
    print(f'"{text}" is probably a less good review ...')
  

In [3]:
#==================================================== 
def split_to_X_y(df, target_column):
    TRAINING_FEATURES = df.columns[df.columns != target_column]
    TARGET_FEATURE = target_column
    
    X=df[TRAINING_FEATURES]
    y=df[TARGET_FEATURE]
    
    return X,y
#==================================================== 
def split_to_train_and_test(X, y, test_ratio, rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    
    return X_train, X_test, y_train,y_test
#==================================================== 
def manipulate_feature_vector(X):
    #Encode
    le1 = LabelEncoder()
    X['company_name'] = le1.fit_transform(X["company_name"])
    EncodedCompanies = dict(zip(le1.classes_, le1.transform(le1.classes_)))

    return X, EncodedCompanies
#==================================================== 
def evo_met(y_test , y_pred,name):   
    evo = metrics.confusion_matrix(y_test , y_pred)
    df_confusion = pd.DataFrame(evo , index=['Actual Negative','Actual Positive'],columns=['Predicted Negative','Predicted Positive'])
    print("confusion matrix of",name,"is:\n")
    display(df_confusion)
#==================================================== 
def plotplot(machine,train,test,name,color1,color2):
  
  font = {'size': 25}
  plt.rc('font', **font)

  x = np.arange(len(machine))  # the label locations
  width = 0.2  # the width of the bars

  fig, ax = plt.subplots(figsize=(30,15))
  rects1 = ax.bar(x - width/2, train, width, label='Train',color=color1)
  rects2 = ax.bar(x + width/2, test, width, label='Test',color=color2)

  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_ylabel('Scores',fontsize=50)
  ax.set_title(name,fontsize=50)
  ax.set_xticks(x)
  ax.set_xticklabels(machine,fontsize=30)
  ax.legend(fontsize=30,loc="lower left")

  def autolabel(rects):
      """Attach a text label above each bar in *rects*, displaying its height."""
      for rect in rects:
          height = rect.get_height()
          ax.annotate('{}'.format(height),
                      xy=(rect.get_x() + rect.get_width() / 2, height),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')


  autolabel(rects1)
  autolabel(rects2)

  fig.tight_layout()
  plt.show()


In [4]:
df = pd.read_csv('/content/drive/Othercomputers/ה-מחשב נייד שלי/Desktop/Python Project/Data/whole_economy_data.csv', index_col=0) 

df

,company_name,year_of_employment,year_of_resignation,ranking,review_essence,disadvantages,advantages,workplace_seniority
0,סופר פארם,2019,2019,2.0,"ניצול העובד עד המקסימום, תמורה לא בהתאם","שכר נמוך , הנהלה של החברה לא תומך ולא חושב על ...","הנחה בעת קנייה של מוצרים, ימי כיף ,משמרות נוחות ,",0
1,ללין,2016,2016,3.0,חברה גדולה טובה ומוצלחת .,שעות עבודה מעטות. שעות העבודה בשבת הן הין 6-8 ...,האווירה נעימה ומדהימה. שירות הלקוחות מעולה. כי...,0
2,טסקום,2016,2016,3.1,חברה טובה ואיכותית לעבודה,לא מצאתי שום חסרונות,ניהול טוב יחס טוב,0
3,תן ביס,2019,2019,3.0,ממש נהנה מכל רגע בו אני מגיע לעבודה,אין לי דבר אחד רע להגיד עליהם!,אהבתי את האווירה וזה חיזק אותי נפשית,0
4,ארנסט אנד יאנג,2016,2016,3.2,ממליץ בחום לרזומה מצויין,"אין הרבה חסרונות , אך אין גיבוש חברתי",נעים לסביבת עבודה איכותית,0
...,...,...,...,...,...,...,...,...
55979,פריקור,1990,2021,4.4,"ידע, טכנולוגיה, רגולציה והכרות עם שלל לקוחות",עבודה בשעות חריגות ולעיתים תחת לחץ זמנים,"עסק שהוא משפחה, מקום אידיאלי לעבודה עם צוות מק...",31
55980,משרד החינוך,1989,2020,4.1,תחושה של שליחות לאורך כל הדרך,אין הרבה לאן להתקדםהשתלמויות נתונות לחא חופש ב...,חופשיםשעות נוחותשנת שבתוןעניין רבהשתלמויות כמע...,31
55981,חברת החשמל,1989,2020,4.3,חברה שמתחדשת כל הזמן.,עדיין יש תפיסה של פעם. קידום בגלל וותק ולא כיש...,חברה שהיא מספיק גדולה כדי למצוא תחומי עניין שו...,31
55982,"קמ""ג",1989,2021,4.5,"הרגשת שליחות, אווירה משפחתית",עובדי מדינה עם כל המשתמע מכך,"ביטחון תעסוקתי, אווירה משפחתית, שכר הולם, עבוד...",32


In [5]:
#Check if the review contains a negative word, if so, we will classify it as a potential for a negative review, (0) otherwise, we will classify it as positive (1)
negative_review_potential = []

for sentence in df['review_essence']:
  if re.search(r'\bלא\b', sentence):
    negative_review_potential.append(1)
  else:
    negative_review_potential.append(0)

    
df['negative_review_potential'] = negative_review_potential

#==================================================== 
#Check whether the rating passes rating 3.9 and not include keywords, We classify as satisfied (1) otherwise, we will classify as dissatisfied (0)
pos_or_neg = []

for rank,sentence in zip(df['ranking'],df['review_essence']):
  if (re.search(r'\bלא\b', sentence) or
      re.search(r'\bאין לאן\b', sentence) or
      re.search(r'\bלהתרחק\b', sentence) or
      re.search(r'\bשלא\b', sentence) or
      re.search(r'\bגרוע\b', sentence) or
      re.search(r'\bגרועה\b', sentence) or
      rank <= 2.2) and not re.search(r'\bלא קשה\b', sentence) and not rank>=3.9:
    pos_or_neg.append(0)
  else:
    pos_or_neg.append(1)

df['pos_or_neg'] = pos_or_neg


df

,company_name,year_of_employment,year_of_resignation,ranking,review_essence,disadvantages,advantages,workplace_seniority,negative_review_potential,pos_or_neg
0,סופר פארם,2019,2019,2.0,"ניצול העובד עד המקסימום, תמורה לא בהתאם","שכר נמוך , הנהלה של החברה לא תומך ולא חושב על ...","הנחה בעת קנייה של מוצרים, ימי כיף ,משמרות נוחות ,",0,1,0
1,ללין,2016,2016,3.0,חברה גדולה טובה ומוצלחת .,שעות עבודה מעטות. שעות העבודה בשבת הן הין 6-8 ...,האווירה נעימה ומדהימה. שירות הלקוחות מעולה. כי...,0,0,1
2,טסקום,2016,2016,3.1,חברה טובה ואיכותית לעבודה,לא מצאתי שום חסרונות,ניהול טוב יחס טוב,0,0,1
3,תן ביס,2019,2019,3.0,ממש נהנה מכל רגע בו אני מגיע לעבודה,אין לי דבר אחד רע להגיד עליהם!,אהבתי את האווירה וזה חיזק אותי נפשית,0,0,1
4,ארנסט אנד יאנג,2016,2016,3.2,ממליץ בחום לרזומה מצויין,"אין הרבה חסרונות , אך אין גיבוש חברתי",נעים לסביבת עבודה איכותית,0,0,1
...,...,...,...,...,...,...,...,...,...,...
55979,פריקור,1990,2021,4.4,"ידע, טכנולוגיה, רגולציה והכרות עם שלל לקוחות",עבודה בשעות חריגות ולעיתים תחת לחץ זמנים,"עסק שהוא משפחה, מקום אידיאלי לעבודה עם צוות מק...",31,0,1
55980,משרד החינוך,1989,2020,4.1,תחושה של שליחות לאורך כל הדרך,אין הרבה לאן להתקדםהשתלמויות נתונות לחא חופש ב...,חופשיםשעות נוחותשנת שבתוןעניין רבהשתלמויות כמע...,31,0,1
55981,חברת החשמל,1989,2020,4.3,חברה שמתחדשת כל הזמן.,עדיין יש תפיסה של פעם. קידום בגלל וותק ולא כיש...,חברה שהיא מספיק גדולה כדי למצוא תחומי עניין שו...,31,0,1
55982,"קמ""ג",1989,2021,4.5,"הרגשת שליחות, אווירה משפחתית",עובדי מדינה עם כל המשתמע מכך,"ביטחון תעסוקתי, אווירה משפחתית, שכר הולם, עבוד...",32,0,1


In [6]:
#Filter data for reviews only
df_review_label = df.drop(['company_name','year_of_employment','year_of_resignation','ranking','workplace_seniority','negative_review_potential'], axis=1)

df_review_label


,review_essence,disadvantages,advantages,pos_or_neg
0,"ניצול העובד עד המקסימום, תמורה לא בהתאם","שכר נמוך , הנהלה של החברה לא תומך ולא חושב על ...","הנחה בעת קנייה של מוצרים, ימי כיף ,משמרות נוחות ,",0
1,חברה גדולה טובה ומוצלחת .,שעות עבודה מעטות. שעות העבודה בשבת הן הין 6-8 ...,האווירה נעימה ומדהימה. שירות הלקוחות מעולה. כי...,1
2,חברה טובה ואיכותית לעבודה,לא מצאתי שום חסרונות,ניהול טוב יחס טוב,1
3,ממש נהנה מכל רגע בו אני מגיע לעבודה,אין לי דבר אחד רע להגיד עליהם!,אהבתי את האווירה וזה חיזק אותי נפשית,1
4,ממליץ בחום לרזומה מצויין,"אין הרבה חסרונות , אך אין גיבוש חברתי",נעים לסביבת עבודה איכותית,1
...,...,...,...,...
55979,"ידע, טכנולוגיה, רגולציה והכרות עם שלל לקוחות",עבודה בשעות חריגות ולעיתים תחת לחץ זמנים,"עסק שהוא משפחה, מקום אידיאלי לעבודה עם צוות מק...",1
55980,תחושה של שליחות לאורך כל הדרך,אין הרבה לאן להתקדםהשתלמויות נתונות לחא חופש ב...,חופשיםשעות נוחותשנת שבתוןעניין רבהשתלמויות כמע...,1
55981,חברה שמתחדשת כל הזמן.,עדיין יש תפיסה של פעם. קידום בגלל וותק ולא כיש...,חברה שהיא מספיק גדולה כדי למצוא תחומי עניין שו...,1
55982,"הרגשת שליחות, אווירה משפחתית",עובדי מדינה עם כל המשתמע מכך,"ביטחון תעסוקתי, אווירה משפחתית, שכר הולם, עבוד...",1


In [7]:
#Reducing some of the data for balance
filter_rows = df_review_label.index[df['pos_or_neg']==1]
df_review_label.drop(filter_rows[:int(len(filter_rows) * 0.70)], inplace=True)
df_review_label

,review_essence,disadvantages,advantages,pos_or_neg
0,"ניצול העובד עד המקסימום, תמורה לא בהתאם","שכר נמוך , הנהלה של החברה לא תומך ולא חושב על ...","הנחה בעת קנייה של מוצרים, ימי כיף ,משמרות נוחות ,",0
5,טוב כעבודה זמנית,קשה לפתח קריירה,"קל להתקבל, שעות נוחות, פרויקטים שונים ואפשר לע...",0
9,לא טוב חברה מנוונת ללא סיפוק,בורג קטן ללא יכולת השפעה,אין לי מה לציין בהקשר הזה,0
14,"תפקיד ראשוני לקבל נסיון כ ""כלכלן"" ולעבור הלאה.","לא מתמקצעים,לחץ בעבודה, מבחינת המקום ניתן לעבו...","הכותרת של התפקיד יכולה להיות כלכלן,לכן יש לנצל...",0
21,לדעתי לא יציבה כל כך נמכרה והוחלפו חלק גדול מה...,לא מפתחים את העובדים יותר מדי,פשוטה לרוב האנשים,0
...,...,...,...,...
55979,"ידע, טכנולוגיה, רגולציה והכרות עם שלל לקוחות",עבודה בשעות חריגות ולעיתים תחת לחץ זמנים,"עסק שהוא משפחה, מקום אידיאלי לעבודה עם צוות מק...",1
55980,תחושה של שליחות לאורך כל הדרך,אין הרבה לאן להתקדםהשתלמויות נתונות לחא חופש ב...,חופשיםשעות נוחותשנת שבתוןעניין רבהשתלמויות כמע...,1
55981,חברה שמתחדשת כל הזמן.,עדיין יש תפיסה של פעם. קידום בגלל וותק ולא כיש...,חברה שהיא מספיק גדולה כדי למצוא תחומי עניין שו...,1
55982,"הרגשת שליחות, אווירה משפחתית",עובדי מדינה עם כל המשתמע מכך,"ביטחון תעסוקתי, אווירה משפחתית, שכר הולם, עבוד...",1


In [8]:
#Test, Train
trainDf = df_review_label.sample(frac =.80)
df_review_label.drop(trainDf.index, inplace=True)

testDf = df_review_label


# **Pipeline Classification**

In [9]:
#Classification by text
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('norm', preprocessing.Normalizer(norm='l1')),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=55,
                          max_iter=5, tol=None)),
])

In [10]:
text_clf.fit(trainDf.review_essence, trainDf.pos_or_neg)

Pipeline(steps=[('vect', CountVectorizer()), ('norm', Normalizer(norm='l1')),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=55,
                               tol=None))])

In [11]:
#Predict on train
y_pred_train = text_clf.predict(trainDf.review_essence)
np.mean(y_pred_train == trainDf.pos_or_neg)


0.8084407249576423

In [12]:
#Predict on test
y_pred_test=text_clf.predict(testDf.review_essence)
np.mean(y_pred_test == testDf.pos_or_neg)

0.8188539741219963

In [13]:
#Prediction
text = text_clf.predict([''])

In [14]:
#Scores by evo matrix, f1-score, recall, precision, support
print(metrics.classification_report(testDf.pos_or_neg, y_pred_test, target_names=['negative', 'positive']))
evo_met(testDf.pos_or_neg, y_pred_test,'Pipeline train')

              precision    recall  f1-score   support

    negative       0.97      0.61      0.75      2146
    positive       0.76      0.99      0.86      2723

    accuracy                           0.82      4869
   macro avg       0.87      0.80      0.80      4869
weighted avg       0.85      0.82      0.81      4869

confusion matrix of Pipeline train is:



,Predicted Negative,Predicted Positive
Actual Negative,1300,846
Actual Positive,36,2687


In [48]:
#Examples of model predictions
text_prediction('חברה מדהימה עם אופק קידום', text_clf)
text_prediction('לא ממליץ על החברה', text_clf)
text_prediction('לא רואה בחברה הזו אופק קידומי, מה גם שהשכר נמוך יחסית', text_clf)
text_prediction('בהחלט חברה עם עתיד', text_clf)

"['חברה מדהימה עם אופק קידום']" is probably a good review
"['לא ממליץ על החברה']" is probably a less good review ...
"['לא רואה בחברה הזו אופק קידומי, מה גם שהשכר נמוך יחסית']" is probably a less good review ...
"['בהחלט חברה עם עתיד']" is probably a good review


In [16]:
###Functions###
#=============================================
def split_to_train_and_test(X, y, test_ratio, rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    
    return X_train, X_test, y_train,y_test
#=============================================
def evaluate_performance_1st(y_test,y_predicted):
    evaluate_value = r2_score(y_test,y_predicted)
    return evaluate_value  
#=============================================    
def manipulate_feature_vector(X):
  le1 = LabelEncoder()
  X['company_name'] = le1.fit_transform(X["company_name"])
  EncodedCompanies = dict(zip(le1.classes_, le1.transform(le1.classes_)))

  le2 = LabelEncoder()
  X['review_essence'] = le2.fit_transform(X["review_essence"])
  EncodedReview = dict(zip(le2.classes_, le2.transform(le2.classes_)))

  le3 = LabelEncoder()
  X['advantages'] = le3.fit_transform(X["advantages"])
  EncodedReview = dict(zip(le3.classes_, le3.transform(le3.classes_)))

  le4 = LabelEncoder()
  X['disadvantages'] = le4.fit_transform(X["disadvantages"])
  EncodedReview = dict(zip(le4.classes_, le4.transform(le4.classes_)))

  return X

In [17]:
df.ranking = df['ranking']*5

#Round ranking, float to int.
ranking = []
for rank in df['ranking']:
  ranking.append(round(rank))
  
df.ranking = ranking



In [18]:
#Filtering some of the data to create a balance between the different rankings.
filter_rows_ranking_2 = df.index[df['ranking']==2]
df.drop(filter_rows_ranking_2[:int(len(filter_rows_ranking_2) * 0.40)], inplace=True)
filter_rows_ranking_3 = df.index[df['ranking']==3]
df.drop(filter_rows_ranking_3[:int(len(filter_rows_ranking_3) * 0.10)], inplace=True)
filter_rows_ranking_4 = df.index[df['ranking']==4]
df.drop(filter_rows_ranking_4[:int(len(filter_rows_ranking_4) * 0.15)], inplace=True)


# **Split**

In [19]:
X,y = split_to_X_y(df, 'ranking')
X=  manipulate_feature_vector(X)
X_train, X_test, y_train,y_test = train_test_split(X,y, random_state=10, test_size=0.20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

# **Multiclass Logistic Regression**

In [20]:
clf = LogisticRegression()
multi_clf = OneVsRestClassifier(clf)
multi_clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [21]:
ypred_train = multi_clf.predict(X_train)

In [22]:
ypred_test = multi_clf.predict(X_test)

In [23]:
metrics.accuracy_score(y_train, ypred_train)

0.16265880724317325

In [24]:
metrics.accuracy_score(y_test, ypred_test)

0.1661159239081897

# **Multiclass GaussianNB**

In [25]:
gnb = GaussianNB()
multi_clf = OneVsRestClassifier(gnb)
multi_clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=GaussianNB())

In [26]:
ypred_train = multi_clf.predict(X_train)

In [27]:
ypred_test = multi_clf.predict(X_test)

In [28]:
metrics.accuracy_score(y_train, ypred_train)

0.16873199812445575

In [29]:
metrics.accuracy_score(y_test, ypred_test)

0.16924176118603199

# **Multiclass Decision Tree**

In [30]:
decisionTree = DecisionTreeClassifier(max_depth=4, min_samples_split=5)
multi_clf = OneVsRestClassifier(decisionTree)
multi_clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=DecisionTreeClassifier(max_depth=4,
                                                     min_samples_split=5))

In [31]:
ypred_train = multi_clf.predict(X_train)

In [32]:
ypred_test = multi_clf.predict(X_test)

In [33]:
metrics.accuracy_score(y_train, ypred_train)

0.2090561993435595

In [34]:
metrics.accuracy_score(y_test, ypred_test)

0.20353666160578726

# **Multiclass KNN**

In [35]:
knn = KNeighborsClassifier(n_neighbors=11)
multi_clf = OneVsRestClassifier(knn)
multi_clf.fit(X_train,y_train)


OneVsRestClassifier(estimator=KNeighborsClassifier(n_neighbors=11))

In [36]:
ypred_train = multi_clf.predict(X_train)

In [37]:
ypred_test = multi_clf.predict(X_test)

In [38]:
metrics.accuracy_score(y_train, ypred_train)

0.2674660057606002

In [39]:
metrics.accuracy_score(y_test, ypred_test)

0.12137179601679021

In [40]:
resDF=pd.DataFrame({"Actual":y_test,"Predicted":ypred_test})
resDF

,Actual,Predicted
43533,18,14
28022,16,20
40242,14,20
5393,15,15
30419,15,16
...,...,...
9310,17,22
24595,18,24
1988,15,22
1845,18,20


# **Multiclass Random Forest**

In [41]:
rfc = RandomForestClassifier(n_estimators = 71, max_depth=7)
multi_clf = OneVsRestClassifier(rfc)
multi_clf.fit(X_train,y_train)


OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=7,
                                                     n_estimators=71))

In [42]:
ypred_train = multi_clf.predict(X_train)

In [43]:
ypred_test = multi_clf.predict(X_test)

In [44]:
metrics.accuracy_score(y_train, ypred_train)

0.26947551744925985

In [45]:
metrics.accuracy_score(y_test, ypred_test)

0.20362597124229703

In [46]:
resDF=pd.DataFrame({"Actual":y_test,"Predicted":ypred_test})
resDF



,Actual,Predicted
43533,18,20
28022,16,18
40242,14,18
5393,15,15
30419,15,16
...,...,...
9310,17,15
24595,18,25
1988,15,15
1845,18,15


**We made predictions on the satisfaction rating, for results between 1 and 5. We worked with all kinds of methods, the best result was given by Random Forest.**